**1.  Super-resolution: Perform superresolution on the image shown in notebook to enhance its resolution by factor 2. Show a qualitative comparison of original and reconstructed image. (i.e display original image and the image you created side by side) **[3 Marks]****
**2. The above only helps us with a qualitative comparison. Let us now do a quantitative comparison. Compute the below given metrics:  **[1 Marks]**
    - RMSE on predicted v/s ground truth high resolution image
    - Peak SNR on predicted v/s ground truth high resolution image****

**Answer:** Using and Exploring OpenCV:

In [44]:
import cv2 
img = cv2.imread('./dog.jpg')
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [45]:
cv2.imread('./dog.jpg',cv2.IMREAD_COLOR)
cv2.imread('./dog.jpg',cv2.IMREAD_GRAYSCALE)
cv2.imread('./dog.jpg',cv2.IMREAD_UNCHANGED)

array([[[ 37,  51,  79],
        [ 38,  52,  80],
        [ 38,  52,  80],
        ...,
        [ 30,  55,  75],
        [ 30,  54,  76],
        [ 30,  54,  76]],

       [[ 37,  51,  79],
        [ 38,  52,  80],
        [ 38,  52,  80],
        ...,
        [ 30,  55,  75],
        [ 30,  54,  76],
        [ 30,  54,  76]],

       [[ 37,  51,  79],
        [ 38,  52,  80],
        [ 36,  53,  80],
        ...,
        [ 30,  55,  75],
        [ 30,  54,  76],
        [ 30,  54,  76]],

       ...,

       [[ 37, 103, 121],
        [ 37, 103, 121],
        [ 36, 105, 122],
        ...,
        [ 43,  82,  97],
        [ 42,  80,  98],
        [ 42,  80,  98]],

       [[ 37, 103, 121],
        [ 37, 103, 121],
        [ 36, 105, 122],
        ...,
        [ 43,  82,  97],
        [ 43,  81,  99],
        [ 42,  80,  98]],

       [[ 37, 103, 121],
        [ 37, 103, 121],
        [ 36, 105, 122],
        ...,
        [ 44,  83,  98],
        [ 43,  81,  99],
        [ 43,  81,  99]]

In [46]:
import cv2
 #Incompleted code
# read image as grey scale
grey_img = cv2.imread('./dog.jpg')
# save image
status = cv2.imwrite('./dog.jpg',img)
print("Image written to file-system : ",status)

Image written to file-system :  True


In [47]:
# read image
img = cv2.imread('./dog.jpg', cv2.IMREAD_UNCHANGED)
# get dimensions of image
dimensions = img.shape
# height, width, number of channels in image
height = img.shape[0]
width = img.shape[1]
channels = img.shape[2]
print('Image Dimension    : ',dimensions)
print('Image Height       : ',height)
print('Image Width        : ',width)
print('Number of Channels : ',channels)

Image Dimension    :  (1365, 2048, 3)
Image Height       :  1365
Image Width        :  2048
Number of Channels :  3


**Croping the picture to begin with 400*400 resolution image**

In [48]:
import cv2 
  
img = cv2.imread("./dog.jpg") 
print(type(img)) 
  
# Shape of the image 
print("Shape of the image", img.shape) 
  
# [rows, columns] 
crop = img[600:1000, 800:1200]   
  
print("Shape of the cropped image",crop.shape)
cv2.imshow('original', img) 
cv2.imshow('cropped', crop) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

<class 'numpy.ndarray'>
Shape of the image (1365, 2048, 3)
Shape of the cropped image (400, 400, 3)


**Resizing the image to 200*200 resolution**

In [49]:
print('Original Dimensions : ',crop.shape)
 
scale_percent = 50 # percent of original size
width = int(crop.shape[1] * scale_percent / 100)
height = int(crop.shape[0] * scale_percent / 100)
dim = (width, height)
# resize image
resized = cv2.resize(crop, dim, interpolation = cv2.INTER_AREA)
print('Resized Dimensions : ',resized.shape)
cv2.imshow("Resized image", resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

Original Dimensions :  (400, 400, 3)
Resized Dimensions :  (200, 200, 3)


**Model and RFF**

In [50]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn import preprocessing
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics import mean_squared_error
from math import log10, sqrt
from matplotlib import gridspec

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Normalize the images to [-1, 1]
scaler_img = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(resized.reshape(-1, 1))
resized_scaled = scaler_img.transform(resized.reshape(-1, 1)).reshape(resized.shape)
crop_scaled = scaler_img.transform(crop.reshape(-1, 1)).reshape(crop.shape)

# Convert to PyTorch tensors and move to the device
resized_scaled = torch.tensor(resized_scaled).permute(2, 0, 1).float().to(device)  # Shape (3, h, w)
crop_scaled = torch.tensor(crop_scaled).permute(2, 0, 1).float().to(device)        # Shape (3, h, w)

# Create coordinate map function
def create_coordinate_map(img):
    num_channels, height, width = img.shape
    w_coords = torch.arange(width).repeat(height, 1)
    h_coords = torch.arange(height).repeat(width, 1).t()
    w_coords = w_coords.reshape(-1)
    h_coords = h_coords.reshape(-1)
    X = torch.stack([h_coords, w_coords], dim=1).float().to(device)
    Y = img.view(num_channels, height * width).t().float().to(device)
    return X, Y

# Linear Model definition
class LinearModel(nn.Module):
    def __init__(self, in_features, out_features):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        
    def forward(self, x):
        return self.linear(x)

# Create Random Fourier Features
def create_rff_features(X, num_features, sigma):
    rff = RBFSampler(n_components=num_features, gamma=1/(2 * sigma**2))
    X = X.cpu().numpy()
    X = rff.fit_transform(X)
    return torch.tensor(X, dtype=torch.float32).to(device)

# Training function
def train(net, lr, X, Y, epochs, verbose=True):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = net(X)
        loss = criterion(outputs, Y)
        loss.backward()
        optimizer.step()
        if verbose and epoch % 100 == 0:
            print(f"Epoch {epoch} loss: {loss.item():.6f}")
    return loss.item()

# RMSE Calculation
def calculate_rmse(original, predicted):
    # Flatten the (num_channels, height, width) to (num_pixels, num_channels)
    original_flat = original.detach().cpu().numpy().reshape(-1, original.shape[0])  # Flatten height * width and keep channels
    predicted_flat = predicted.detach().cpu().numpy().reshape(-1, predicted.shape[0])
    
    return sqrt(mean_squared_error(original_flat, predicted_flat))

# PSNR Calculation
def psnr(original, predicted):
    original_flat = original.detach().cpu().numpy().reshape(-1, original.shape[0])
    predicted_flat = predicted.detach().cpu().numpy().reshape(-1, predicted.shape[0])
    
    mse = mean_squared_error(original_flat, predicted_flat)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0  # Since images are normalized to [-1, 1]
    psnr_value = 20 * log10(max_pixel / sqrt(mse))
    return psnr_value


# Generate coordinates and pixel values for the 200x200 image
dog_X, dog_Y = create_coordinate_map(resized_scaled)
X_rff = create_rff_features(dog_X, num_features=20000, sigma=0.008)  # Adjust num_features and sigma for better performance

# Define and train the model
net = LinearModel(X_rff.shape[1], 3).to(device)
train(net, lr=0.005, X=X_rff, Y=dog_Y, epochs=1000)

# Predict the pixel values for the 400x400 image
coords_400, _ = create_coordinate_map(crop_scaled)
X_rff_400 = create_rff_features(coords_400, num_features=20000, sigma=0.008)
predicted_Y = net(X_rff_400).view(crop_scaled.shape)

# Calculate RMSE and PSNR
rmse = calculate_rmse(crop_scaled, predicted_Y)
psnr_value = psnr(crop_scaled, predicted_Y)

print(f"RMSE: {rmse}")
print(f"PSNR: {psnr_value}")

def plot_reconstructed_and_original_image(original_img, predicted_img, title=""):
    """
    Compare the original and predicted images.
    """
    num_channels, height, width = original_img.shape

    fig = plt.figure(figsize=(6, 4))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])

    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1])

    # Plot the reconstructed (predicted) image
    ax0.imshow(predicted_img.detach().cpu().numpy().transpose(1, 2, 0))
    ax0.set_title("Reconstructed Image")

    # Plot the original image
    ax1.imshow(original_img.detach().cpu().numpy().transpose(1, 2, 0))
    ax1.set_title("Original Image")

    for a in [ax0, ax1]:
        a.axis("off")

    fig.suptitle(title, y=0.9)
    plt.tight_layout()

plot_reconstructed_and_original_image(crop_scaled, predicted_Y, title="Reconstructed vs Original")

Epoch 0 loss: 0.235375
Epoch 100 loss: 0.144749
Epoch 200 loss: 0.142958
Epoch 300 loss: 0.142957
Epoch 400 loss: 0.142957
Epoch 500 loss: 0.142957
Epoch 600 loss: 0.142957
Epoch 700 loss: 0.142957
Epoch 800 loss: 0.142957
Epoch 900 loss: 0.142957
